In [ ]:
from types import SimpleNamespace
from pprint import pprint
import numpy as np

import anomapy.train.sssn as sssn
import datasets

import pyworld.toolkit.tools.wbutils as wbu
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.plot as vplot

num_episodes = 1


envs = ['BeamRider','Breakout','Enduro','Pong','Qbert','Seaquest','SpaceInvaders']

def load_model(run):
    models, config = wbu.load(run) #load the most recently trained model
    model = models['model.pt'].load(sssn.model(**config))
    config = SimpleNamespace(**config)
    return model, config

def load_data(num_episodes):
    dataset = datasets.dataset('aad.clean.{0}'.format(env))
    dataset.state.transform.to_float().CHW().torch()
    return [d for d in dataset.state.load(num_episodes)]

def encode(env, plot):
    dryruns = sorted([r for r in wbu.dryruns() if env in r])
    run = dryruns[0]
    plot_path = "wandb/{0}/media/{2}-{1}.png".format(run, env, plot)
    model, config = load_model(run)
    episodes = load_data(num_episodes)
    episode = episodes[0]
    return tu.to_numpy(sssn.encode(model, episode)), plot_path

def displacement(z, i):
    return ((z[:-i] - z[i:]) ** 2).sum(1)

def distance(z, i): #sum(|| s_t - s_t+j ||) for all j in {t,...,t+i}
    d = distance(z[:-1], z[1:])
    shape = (d.size - i + 1, i)
    strides = (d.itemsize, d.itemsize)
    return np.lib.stride_tricks.as_strided(d, shape=shape, strides=strides).sum(1)

import numpy as np

def displacement_summary(z): # TODO complexity can be cut in half... 
    z = z[...,np.newaxis]  
    d = ((z.T - z)**2).sum(1)
    s1 = np.diagonal(d,offset=1)[:,np.newaxis]
    sj = d[:-1,1:]
    d = np.triu(np.greater(s1, sj).astype(np.uint8), k=1)
    dmu = d.sum(1).astype(np.float32)
    dmu[:-1] = dmu[:-1] / np.arange(dmu.size-1,0,-1) 

    return dmu, d.sum() * (2 / ((z.shape[0]-1)*(z.shape[0]-2))) #statistic

def real_displacement_summary(z):
    z = z[...,np.newaxis]  
    d = ((z.T - z)**2).sum(1)
    s1 = np.diagonal(d,offset=1)[:,np.newaxis]
    sj = d[:-1,1:]
    d = np.triu(np.maximum(s1 - sj, 0), k=1)
    # mean displacement
    dmu = d.sum(1)
    dmu[:-1] = dmu[:-1] / np.arange(dmu.size-1,0,-1) 
    
    #max displacment
    dmax = np.max(d, axis=1)
    #min displacement
    dmin = np.min(d, axis=1)
    
    return dmu, dmax, dmin



In [ ]:
# plot the false postive rate - when displacement 1 >= displacement j for each state

summaries = {}

for env in envs:
    z, plot_path = encode(env, "normal-FP")
    ds, summary = displacement_summary(z)
    summaries[env] = summary
    plot = J.plot(np.arange(0,ds.size), ds)
    plot.fig.update_layout(margin=dict(t=10,l=10,r=10,b=10), autosize=False, height=200, width=800, showlegend=False)
    plot.fig['layout']['yaxis1'].update(title='', range=[0, 1], dtick=.2, autorange=False)
    plot.fig.write_image(plot_path)

pprint(summaries)
    

In [ ]:
for env in envs:
    z, plot_path = encode(env,  'real-displacement')
    print(z.shape)
    mu,_max,_min = real_displacement_summary(z)
    x = np.arange(0, mu.size)
    plot = J.plot(x, _max)
    plot.fig.update_layout(margin=dict(t=10,l=10,r=10,b=10), autosize=False, height=200, width=800, showlegend=False)
    #plot.fig['layout']['yaxis1'].update(title='', range=[0, 1], dtick=.2, autorange=False)
    #plot.fig.write_image(plot_path)

    
    

In [ ]:
import numpy as np

n = 1000
x = np.arange(0,n)
y = np.random.normal(0,1,size=x.size)

def estd(y, s):
    return np.sqrt(((y - np.mean(y))**s).sum() / (y.size - 1))
    

print(np.std(y))
print(estd(y, 10))


y[np.random.randint(n)] = 8
print(np.std(y)**2)
print(estd(y, 10))

J.plot(x,y)



print(np.std(np.random.uniform(0,1,size=1000)))



In [ ]:
import numpy as np

def displacement_summary(z): # TODO complexity can be cut in half... 
    z = z[...,np.newaxis]  
    d = ((z.T - z)**2).sum(1)
    print(d.shape)
    s1 = np.diagonal(d,offset=1)[:,np.newaxis]
    sj = d[:-1,1:]
    d = np.triu(np.greater(s1, sj).astype(np.uint8), k=1)
    ds = d.sum(1) # number of non-conforming distances for each state
    return ds, ds.sum() * (2 / ((z.shape[0]-1)*(z.shape[0]-2))) #statistic

def real_displacement_summary(z):
    z = z[...,np.newaxis]  
    d = ((z.T - z)**2).sum(1)
    print(d.shape)
    s1 = np.diagonal(d,offset=1)[:,np.newaxis]
    sj = d[:-1,1:]
    d = np.triu(np.maximum(s1 - sj, 0), k=1)
    
    # mean displacement
    dmu = d.sum(1)
    dmu[:-1] = dmu[:-1] / np.arange(dmu.size-1,0,-1) 
    #max displacment
    dmax = np.max(d, axis=1)
    #min displacement
    dmin = np.min(d, axis=1)
    
    return dmu, dmax, dmin

z = np.random.randint(0,10,size=(5,2))
print(z)
real_displacement_summary(z)
